In [4]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
import keras.backend
if keras.backend.backend() != 'tensorflow':
    raise BaseException("This script uses other backend")
else:
    keras.backend.set_image_dim_ordering('th')
    print("Backend ok")

from pythainlp.tokenize import word_tokenize
from pythainlp.word_vector.thai2vec import get_model
from numpy import zeros, array
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from pandas import read_csv
from re import compile
import pickle

word2vec = get_model()

Backend ok


In [6]:
def clean(sentence):
    return sentence.strip()

def remove_tags(sentence):
    regex = compile(r'<\/?[\w-]+>')
    return regex.sub('', sentence)

def remove_dash(sentence):
    regex = compile(r'-')
    return regex.sub('', sentence)

def get_word2idx(tokenized_sentences):
    word2idx ={}
    for sentence in tokenized_sentences:
        for word in sentence:
            if word not in word2idx:
                word2idx[word] = len(word2idx) + 1
    word2idx['UNK'] = len(word2idx)
    return word2idx

def get_index(word, word2idx):
    return word2idx[word] if word in word2idx else word2idx['UNK']

def get_embeddings(word2vec, word2idx, dim=300):
    embeddings = zeros((len(word2idx), dim))
    for (word, i) in word2idx.items():
        if word != 'UNK' and word in word2vec.index2word:
            embeddings[i] = word2vec.word_vec(word)
        else:
            pass
    return embeddings

def get_training_data(tokenized_sentences, classes, classes2, word2idx, maxlen=1000, class_num=4, class_num_two=4):
    train_x = array([[get_index(word, word2idx) for word in sentence] for sentence in tokenized_sentences])
    train_y = [cls for cls in classes]
    train_z = [cls for cls in classes2]

    train_x = pad_sequences(train_x, maxlen=maxlen, dtype='int32', padding='post', truncating='pre', value=0.)
    train_y = array([[1 if i == cls else 0 for i in range(0, class_num)] for cls in train_y])
    train_z = array([[1 if i == cls else 0 for i in range(0, class_num_two)] for cls in train_z])

    return (train_x, train_y, train_z)

def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    r = {}
    with open(filename, 'rb') as f:
        r = pickle.load(f)
    return r

In [7]:
from sklearn.model_selection import train_test_split
mood = []
genre = []


def get_corpus(path):
    global mood, genre
    corpus = read_csv(path)
    corpus = corpus[['sentence', 'mood', 'genre']]
    corpus['sentence'] = corpus['sentence'].apply(clean).apply(remove_tags)
    corpus['tokenized_sentence'] = corpus['sentence'].apply(word_tokenize, engine='deepcut')
    corpus['genre'] = corpus['genre'].apply(remove_dash)
    mood += list(corpus.mood.unique())
    genre += list(corpus.genre.unique())
    return corpus

corpus = get_corpus('NLP-Corpus_yak.csv')
train_corpus = corpus.copy()
test_corpus = corpus.sample(n=7,replace=True).copy()


mood = sorted(list(set(mood)), reverse=True)
genre = sorted(list(set(genre)), reverse=True)
mood2idx = dict(zip(mood, [i for i in range(len(mood))]))
genre2idx = dict(zip(genre, [i for i in range(len(genre))]))

def pos_process(corpus):
    global mood2idx, genre2idx
    corpus['genre'] = corpus['genre'].apply(lambda x: genre2idx[x])
    corpus['mood'] = corpus['mood'].apply(lambda x: mood2idx[x])
    return corpus

train_corpus = pos_process(train_corpus)
test_corpus = pos_process(test_corpus)
corpus = pos_process(corpus)

print(mood, genre, corpus)

['เศร้า', 'สนุก', 'รัก', 'ชิว', 'all'] ['ลูกทุ่ง', 'rock', 'pop', 'hiphop', 'all']                                    sentence  mood  genre  \
0                 เพลงชิว ฟังสบาย จังหวะเบา     3      2   
1                ทำงานอยู่ เพลงสบายฟังหน่อย     3      4   
2            จัดมาสักเพลง สบาย ฟังแล้วเพลิน     3      4   
3      เพลงแบบ ฟังแล้วผ่อนคลาย สบายไปวันนี้     3      2   
4   เหนื่อยแล้วอยากพักผ่อน เพลงมาฟังหย่อนใจ     3      2   
5                       เพลงชิว กำลังสบายใจ     3      2   
6                          แบบเพราะเพลินชิว     3      2   
7                   เพลง เบา ฟังสบายจิบกาแฟ     3      2   
8                      เพลงเพราะชิว ฟังสบาย     3      2   
9            ชิวแบบนี้ พอเพลงฟังเพลิน เพราะ     3      2   
10              เพลงอกหัก เศร้าบาดใจ ฟังช้า     0      2   
11                      เพลงแบบคนโดนเทหน่อย     0      4   
12  เพิ่งโดนเพื่อนหักหลังมา เพลงปลอบใจหน่อย     0      4   
13         ตอนนี้เหงา อยากฟังเพลงอกหักหน่อย     0      4   
14             ง่

In [373]:
display(test_corpus)
display(train_corpus)

,sentence,mood,genre,tokenized_sentence
13,ตอนนี้เหงา อยากฟังเพลงอกหักหน่อย,0,4,"[ตอน, นี้, เหงา, , อยาก, ฟัง, เพล, ง, อก, หัก..."
39,เพลงรักแบบหวาน ฟังเพราะ,2,4,"[เพลง, รัก, แบบ, หวาน, , ฟัง, เพราะ]"
13,ตอนนี้เหงา อยากฟังเพลงอกหักหน่อย,0,4,"[ตอน, นี้, เหงา, , อยาก, ฟัง, เพล, ง, อก, หัก..."
15,โดนเท เพลงแก้เซ็ง,0,4,"[โดน, เท, , เพลง, แก้เซ็ง]"
39,เพลงรักแบบหวาน ฟังเพราะ,2,4,"[เพลง, รัก, แบบ, หวาน, , ฟัง, เพราะ]"
28,เพลงสนุกสนาน ฟังสนุก,1,4,"[เพลง, สนุกสนาน, , ฟัง, สนุก]"
24,เพลงคึกคัก ฟังแล้วต้องเต้น,1,4,"[เพลง, คึกคัก, , ฟัง, แล้ว, ต้อง, เต้น]"


,sentence,mood,genre,tokenized_sentence
0,เพลงชิว ฟังสบาย จังหวะเบา,3,2,"[เพลง, ชิว, , ฟัง, สบาย, , จังหวะ, เบา]"
1,ทำงานอยู่ เพลงสบายฟังหน่อย,3,4,"[ทำ, งาน, อยู่, , เพลง, สบาย, ฟัง, หน่อย]"
2,จัดมาสักเพลง สบาย ฟังแล้วเพลิน,3,4,"[จัด, มา, สัก, เพลง, , สบาย, , ฟัง, แล้ว, เพ..."
3,เพลงแบบ ฟังแล้วผ่อนคลาย สบายไปวันนี้,3,2,"[เพลง, แบบ, , ฟัง, แล้ว, ผ่อนคลาย, , สบาย, ไ..."
4,เหนื่อยแล้วอยากพักผ่อน เพลงมาฟังหย่อนใจ,3,2,"[เหนื่อย, แล้ว, อยาก, พักผ่อน, , เพลง, มา, ฟั..."
5,เพลงชิว กำลังสบายใจ,3,2,"[เพลง, ชิว, , กำลัง, สบายใจ]"
6,แบบเพราะเพลินชิว,3,2,"[แบบ, เพราะ, เพลินชิว]"
7,เพลง เบา ฟังสบายจิบกาแฟ,3,2,"[เพลง, , เบา, , ฟัง, สบาย, จิบ, กาแฟ]"
8,เพลงเพราะชิว ฟังสบาย,3,2,"[เพลง, เพราะ, ชิว, , ฟัง, สบาย]"
9,ชิวแบบนี้ พอเพลงฟังเพลิน เพราะ,3,2,"[ชิว, แบบ, นี้, , พอ, เพลง, ฟัง, เพลิน, , เพ..."


In [374]:
word2idx = get_word2idx(corpus['tokenized_sentence'].values)

In [375]:
#Use in development only
save_object(word2idx, 'models/word2idx.pkl')
word2idx = load_object('models/word2idx.pkl')

In [376]:
embeddings = get_embeddings(word2vec, word2idx)

In [377]:
# train_x, train_y, train_z = get_training_data(train_corpus['tokenized_sentence'].values, train_corpus['mood'].values, train_corpus['genre'].values, word2idx)
train_x, train_y, train_z = get_training_data(corpus['tokenized_sentence'].values, corpus['mood'].values, corpus['genre'].values, word2idx)
test_x, test_y, test_z = get_training_data(test_corpus['tokenized_sentence'].values, test_corpus['mood'].values, test_corpus['genre'].values, word2idx)

In [378]:
print(train_x.shape, train_y.shape, train_z.shape)
print(test_x.shape, test_y.shape, test_z.shape)

(48, 1000) (48, 4) (48, 4)
(7, 1000) (7, 4) (7, 4)


In [379]:
from keras.models import Model
from keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dropout, Dense, concatenate
from keras.regularizers import l2
from keras.constraints import max_norm
from keras.optimizers import Adam

def get_classification_model(input_size, vocab_size, embeddings, embedding_dimension=300, class_num=4, class_num_two=4):
    inp = Input(shape=(input_size,))
    x = Embedding(vocab_size, embedding_dimension, weights=[embeddings], trainable=True)(inp)

    convs = []
    kernel_sizes = [3,4,5]
    for kernel_size in kernel_sizes:
        conv = Conv1D(100, kernel_size, activation='relu')(x)
        pool = GlobalMaxPooling1D()(conv)
        convs.append(pool)
    
    x = concatenate(convs)
    x = Dropout(0.5)(x)
    x = Dense(100, activation='relu', kernel_regularizer=l2(0.01), kernel_constraint=max_norm(3))(x)
    out = Dense(class_num, activation='softmax', name='mood')(x)
    x = Dropout(0.5)(x)
    x = Dense(100, activation='relu', kernel_regularizer=l2(0.01), kernel_constraint=max_norm(3))(x)
    aux_out = Dense(class_num_two, activation='softmax', name='genre')(x)
    
    model = Model(inputs=inp, outputs=[out, aux_out])
    model.summary()
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['categorical_accuracy'], loss_weights=[1., 0.2])
    return model

In [357]:
music_model = get_classification_model(train_x.shape[1], len(word2idx), embeddings)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_21 (Embedding)        (None, 1000, 300)    32400       input_21[0][0]                   
__________________________________________________________________________________________________
conv1d_67 (Conv1D)              (None, 998, 100)     90100       embedding_21[0][0]               
__________________________________________________________________________________________________
conv1d_68 (Conv1D)              (None, 997, 100)     120100      embedding_21[0][0]               
__________________________________________________________________________________________________
conv1d_69 

In [358]:
weight_path = 'models/music_weights.h5'

callbacks_list_model = [
    ModelCheckpoint(
        weight_path,
        monitor = "val_mood_categorical_accuracy",
        mode = 'max',
        verbose = 1,
        save_best_only = True,
        save_weights_only = True,
    )   
]

music_model.fit(train_x, [train_y, train_z], batch_size=128, epochs=10, verbose=1, validation_data=(test_x, [test_y, test_z]), callbacks=callbacks_list_model)

Train on 48 samples, validate on 7 samples
Epoch 1/10
48/48 [==============================] - 4s 80ms/step - loss: 3.9298 - mood_loss: 1.2738 - genre_loss: 0.7436 - mood_categorical_accuracy: 0.2708 - genre_categorical_accuracy: 0.1042 - val_loss: 3.6928 - val_mood_loss: 1.0239 - val_genre_loss: 0.9265 - val_mood_categorical_accuracy: 0.7143 - val_genre_categorical_accuracy: 0.0000e+00

Epoch 00001: val_mood_categorical_accuracy improved from -inf to 0.71429, saving model to models/music_weights.h5
Epoch 2/10
48/48 [==============================] - 2s 51ms/step - loss: 3.6647 - mood_loss: 1.0534 - genre_loss: 0.6384 - mood_categorical_accuracy: 0.3958 - genre_categorical_accuracy: 0.1667 - val_loss: 3.4697 - val_mood_loss: 0.8457 - val_genre_loss: 0.8221 - val_mood_categorical_accuracy: 0.7143 - val_genre_categorical_accuracy: 0.1429

Epoch 00002: val_mood_categorical_accuracy did not improve
Epoch 3/10
48/48 [==============================] - 3s 56ms/step - loss: 3.5663 - mood_loss:

In [359]:
music_model.load_weights(weight_path)

In [360]:
from operator import itemgetter
from numpy import apply_along_axis

def prob2class(probs):
    return apply_along_axis(lambda x: max(enumerate(x), key=itemgetter(1))[0], 1, probs)

In [361]:
pred_y, pred_z = music_model.predict(test_x)
class_y = prob2class(test_y)
class_z = prob2class(test_z)
pred_y = prob2class(pred_y)
pred_z = prob2class(pred_z)

In [362]:
display(class_y)
display(pred_y)

display(class_z)
display(pred_z)

array([2, 2, 3, 0, 1, 3, 3])

array([2, 2, 3, 1, 1, 3, 3])

array([2, 0, 2, 0, 0, 0, 2])

array([2, 2, 2, 2, 2, 2, 2])

In [363]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

def evaluate(test_y, pred_y, labels=[i for i in range(0,10)]):
    f1 = f1_score(test_y, pred_y, average='macro', labels=labels)
    accuracy = accuracy_score(test_y, pred_y)
    precision = precision_score(test_y, pred_y, average='macro', labels=labels)
    recall = recall_score(test_y, pred_y, average='macro', labels=labels)
    return f1, accuracy, precision, recall

display(evaluate(class_y, pred_y))
display(evaluate(class_z, pred_z))
# display(*((class2name(cls), class2name(pred)) for cls, pred in zip(class_y, pred_y) if cls != pred))

/Users/fm_index/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/fm_index/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/fm_index/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/fm_index/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(0.26666666666666666, 0.8571428571428571, 0.25, 0.29999999999999999)

(0.059999999999999998,
 0.42857142857142855,
 0.042857142857142858,
 0.10000000000000001)